In [ ]:
import torch
from tqdm.notebook import tqdm

# plotting
import plotly.io as pio
pio.renderers.default = "vscode"

if torch.cuda.is_available:
  print('CUDA available!')
else:
  print('CUDA not available!')

In [ ]:
def generate_data(frac_train, seed):
    torch.manual_seed(seed)
    data = torch.randint(0, 1000, (1000, 2))
    labels = data[:, 0] * data[:, 1]
    

In [ ]:


train_data, test_data, train_labels, test_labels = generate_train_test_data(frac_train, seed)

train_losses = []
test_losses = []
train_accs = []
test_accs = []

print('Initializing model...')
model = torch.nn.Sequential(
    torch.nn.Linear(2, 2),
    torch.nn.ReLU(),
)
model.cuda()

num_epochs = 1000
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=1)

print('Training...')
for epoch in tqdm(range(num_epochs)):
    train_logits = model(train_data)
    train_loss = loss_fn(train_logits, train_labels)
    train_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    train_losses.append(train_loss.item())
    with torch.inference_mode():
        metric = metrics.get_metrics(model, train_logits, train_loss)
        test_losses.append(metric['test_loss'])
        train_accs.append(metric['train_acc'])
        test_accs.append(metric['test_acc'])
        wandb.log(metric)

    if epoch%1000 == 0:
        print(f"Epoch:{epoch}, Train: L: {train_losses[-1]:.6f} A: {train_accs[-1]*100:.4f}%, Test: L: {test_losses[-1]:.6f} A: {test_accs[-1]*100:.4f}%")

lines([train_losses, test_losses], log_y=True, labels=['train loss', 'test loss'], save=f"{task_dir}/loss.png")
lines([train_accs, test_accs], log_y=False, labels=['train acc', 'test acc'], save=f"{task_dir}/acc.png")
torch.save(model.state_dict(), f"{task_dir}/model.pt")
